In [125]:
import openai
import pandas as pd
import sqlite3
import csv


In [126]:
sqlite_dir='sqlite_db/'

In [127]:
conn = sqlite3.connect('toy.db')  
c = conn.cursor() # 

# Create table - CLIENTS
# c.execute('''CREATE TABLE test_table
#              ([id] INTEGER PRIMARY KEY, [name] text, [email] text, [joining_date] date, [salary] integer)''')

c.execute('''DROP TABLE IF EXISTS portfolios''')
conn.commit()

c.execute('''DROP TABLE IF EXISTS portfolio_holdings''')
conn.commit()

c.execute('''DROP TABLE IF EXISTS stock_prices''')
conn.commit()

# Create table - PORTFOLIOS
c.execute('''CREATE TABLE IF NOT EXISTS portfolios (portfolio_id INT, portfolio VARCHAR(50), portfolio_owner VARCHAR(50), age INT)''')
conn.commit()

file = open(f"{sqlite_dir}/portfolios.csv",encoding='utf-8-sig')
contents = csv.reader(file)
insert_records = "INSERT INTO portfolios (portfolio_id,portfolio, portfolio_owner,age) VALUES(?, ?, ?, ?)"
c.executemany(insert_records, contents)

# Create table - PORTFOLIO HOLDINGS
c.execute('''CREATE TABLE IF NOT EXISTS portfolio_holdings (portfolio_id INT, date DATE,ticker VARCHAR(10),shares INT)''')
conn.commit()

file = open(f"{sqlite_dir}/portfolio_holdings.csv",encoding='utf-8-sig')
contents = csv.reader(file)
insert_records = "INSERT INTO portfolio_holdings (portfolio_id,date,ticker,shares) VALUES(?, ?,?,?)"
c.executemany(insert_records, contents)

# Create table - STOCK PRICES
c.execute('''CREATE TABLE IF NOT EXISTS stock_prices (date DATE,ticker VARCHAR(10),price NUMERIC)''')
conn.commit()

file = open(f"{sqlite_dir}/prices_melted.csv",encoding='utf-8-sig')
contents = csv.reader(file)
insert_records = "INSERT INTO stock_prices (date,ticker,price) VALUES(?, ?,?)"
c.executemany(insert_records, contents)

conn.commit()
conn.close()

In [128]:
conn = sqlite3.connect("toy.db")

cur = conn.cursor()
# cur.execute("SELECT portfolio FROM portfolios p JOIN portfolio_holdings ph ON p.portfolio_id = ph.portfolio_id WHERE ph.ticker = 'TSLA' AND ph.date = '2024-08-01';")
cur.execute(" SELECT price FROM stock_prices WHERE ticker = 'GOOG' AND date = '2024-01-31'")
rows = cur.fetchall()

for row in rows:
  print(row)

conn.close()

(141.639,)


In [129]:

# Set your OpenAI API key here
openai.api_key = 'sk-proj-71b78BiQ7GEK7mgEY0IWT3BlbkFJeP8FXPxN07go2mVgcvsq'

schema="""CREATE TABLE portfolios (portfolio_id INT, portfolio VARCHAR(50),portfolio_owner (VARCHAR(50), age INT));
CREATE TABLE portfolio_holdings (portfolio_id INT, date DATE,ticker VARCHAR(10),shares INT);
CREATE TABLE stock_prices (date DATE,ticker VARCHAR(10),price NUMERIC);"""

# System prompt specifically tailored for code writing
system_prompt = """You are a text to SQL query translator. Use SQLite dialect only. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA only.
SCHEMA:
{schema}"""

def get_sql_from_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500
    )
    return response['choices'][0]['message']['content']

def load_and_call_function(function_code, function_name, *args, **kwargs):
    # Execute the function code to define it in the global scope
    print("trying exec")
    exec(function_code, globals())
    print("completed exec")
    
    # Call the function by name
    print("trying actual function")
    result = globals()[function_name](*args, **kwargs)
    print("completed function")
    
    return result


In [130]:
def get_sqlite_answer(sql_string):

  conn = sqlite3.connect("toy.db")
  cur = conn.cursor()
  try:
    cur.execute(sql_string)
    rows = cur.fetchall()
    answer=rows[0][0]
    conn.close()
    if answer is None:
      return 0
    else:
      return answer
  except sqlite3.OperationalError as e:
      # Handle specific SQL execution errors
      print(f"SQL execution error: {e}")
      return "error"
  except sqlite3.DatabaseError as e:
      # Handle general database errors
      print(f"Database error: {e}")
      return "error"
  except Exception as e:
      # Handle other exceptions
      print(f"An unexpected error occurred: {e}")
      return "error"
  finally:
    # Ensure the connection is closed
    if conn:
      conn.close()
  conn.close()

In [131]:
questions_df=pd.read_csv('sqlite_db/sql_questions_for_code_execution.csv')

In [132]:
def convert_to_int_or_return(value):
    try:
        float_value = float(value)
        # Attempt to convert the value to an integer
        return int(float_value)
    except ValueError:
        # If a ValueError occurs, return the original value
        return value

In [133]:

qa_easy_success_rate=[]
qa_med_hard_success_rate=[]
easy_error_count=0
med_hard_error_count=0

for index,row in questions_df.iterrows():
    
    prompt_template = "Please write a SQL query that answers this question: {question}. Use this schema: {schema}. Ensure that the response is valid SQL and does not include any extra text."

    # example question request
    question = row['question']
    actual_answer=convert_to_int_or_return(row['answer'])
    difficulty=row['difficulty']
    prompt = prompt_template.format(question=question,schema=schema)
    print(prompt)

    # Get the sql response from GPT-4
    sql_code = get_sql_from_gpt(prompt)
    print("Received sql code:\n", sql_code)
    print("ground truth answer: ",actual_answer)
    result=convert_to_int_or_return(get_sqlite_answer(sql_code))
    print("generated answer: ",result)
    if actual_answer==result:
      if difficulty=='easy':
        qa_easy_success_rate.append(1)
      else:
        qa_med_hard_success_rate.append(1)
    elif result=="error":
      if difficulty=='easy':
        qa_easy_success_rate.append(0)
        easy_error_count+=1
      else:
        qa_med_hard_success_rate.append(0)
        med_hard_error_count+=1
    else:
      if difficulty=='easy':
        qa_easy_success_rate.append(0)
      else:
        qa_med_hard_success_rate.append(0)
    if len(qa_easy_success_rate)>0:
      qa_easy_accuracy = sum(qa_easy_success_rate)/len(qa_easy_success_rate)
      print(f"Easy Q&A Accuracy: {qa_easy_accuracy*100:.2f}%")
      print("Easy SQL Execution Error Rate: ",easy_error_count/len(qa_easy_success_rate))
    if len(qa_med_hard_success_rate)>0:
      qa_med_hard_accuracy = sum(qa_med_hard_success_rate)/len(qa_med_hard_success_rate)
      print(f"Med/Hard Q&A Accuracy: {qa_med_hard_accuracy*100:.2f}%")
      print("Med/Hard SQL Execution Error Rate: ",med_hard_error_count/len(qa_med_hard_success_rate))

qa_easy_accuracy = sum(qa_easy_success_rate)/len(qa_easy_success_rate)
qa_med_hard_accuracy = sum(qa_med_hard_success_rate)/len(qa_med_hard_success_rate)

print(f"Easy Q&A Accuracy: {qa_easy_accuracy*100:.2f}%")
print(f"Easy SQL Execution Error Rate: {(easy_error_count/len(qa_easy_success_rate))*100:.2f}%")

print(f"Med/Hard Q&A Accuracy: {qa_med_hard_accuracy*100:.2f}%")
print(f"Med/Hard SQL Execution Error Rate: {(med_hard_error_count/len(qa_med_hard_success_rate))*100:.2f}%")

Please write a SQL query that answers this question: How many different portfolios are there?. Use this schema: CREATE TABLE portfolios (portfolio_id INT, portfolio VARCHAR(50),portfolio_owner (VARCHAR(50), age INT));
CREATE TABLE portfolio_holdings (portfolio_id INT, date DATE,ticker VARCHAR(10),shares INT);
CREATE TABLE stock_prices (date DATE,ticker VARCHAR(10),price NUMERIC);. Ensure that the response is valid SQL and does not include any extra text.
Received sql code:
 SELECT COUNT(DISTINCT portfolio_id) FROM portfolios;
ground truth answer:  5
generated answer:  5
Easy Q&A Accuracy: 100.00%
Easy SQL Execution Error Rate:  0.0
Please write a SQL query that answers this question: What is the id of portfolioA?. Use this schema: CREATE TABLE portfolios (portfolio_id INT, portfolio VARCHAR(50),portfolio_owner (VARCHAR(50), age INT));
CREATE TABLE portfolio_holdings (portfolio_id INT, date DATE,ticker VARCHAR(10),shares INT);
CREATE TABLE stock_prices (date DATE,ticker VARCHAR(10),pric